In [ ]:
import pandas as pd
import numpy as np

data=pd.read_csv("/content/USA_Housing.csv")
data.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
0,79545.45857,5.682861,7.009188,4.09,23086.80050,1.059034e+06
1,79248.64245,6.002900,6.730821,3.09,40173.07217,1.505891e+06
2,61287.06718,5.865890,8.512727,5.13,36882.15940,1.058988e+06
3,63345.24005,7.188236,5.586729,3.26,34310.24283,1.260617e+06
4,59982.19723,5.040555,7.839388,4.23,26354.10947,6.309435e+05


Q1: K-Fold Cross Validation for Multiple Linear Regression (Least Square Error Fit)  
Download the dataset regarding USA House Price Prediction from the following link:  
https://drive.google.com/file/d/1O_NwpJT-8xGfU_-3llUl2sgPu0xllOrX/view?usp=sharing  
Load the dataset and Implement 5- fold cross validation for multiple linear regression
(using least square error fit).  
Steps:  
a) Divide the dataset into input features (all columns except price) and output variable  
(price)

In [ ]:
x=data.drop('Price',axis=1)
y=data['Price']

b) Scale the values of input features.  


In [ ]:
import sklearn.preprocessing as preprocessing
x=preprocessing.scale(x)
print(x)

[[ 1.02865969 -0.29692705  0.02127433  0.08806222 -1.31759867]
 [ 1.00080775  0.02590164 -0.25550611 -0.72230146  0.40399945]
 [-0.68462915 -0.11230283  1.5162435   0.93084045  0.07240989]
 ...
 [-0.48723454  1.28447022 -2.17026949 -1.50025059 -0.29193658]
 [-0.05459152 -0.44669439  0.14154061  1.18205319  0.65111608]
 [-0.28831272  0.01521477 -0.19434166  0.07185495  1.04162464]]


c) Divide input and output features into five folds.  

In [ ]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=5,shuffle=True,random_state=42)
print(kf)

KFold(n_splits=5, random_state=42, shuffle=True)



d) Run five iterations, in each iteration consider one-fold as test set and remaining
four sets as training set. Find the beta (𝛽) matrix, predicted values, and R2_score
for each iteration using least square error fit.  


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

beta_matrices = []
predicted_values = []
r2_scores = []

for train_index, test_index in kf.split(x):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model = LinearRegression()
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)

    beta_matrices.append(model.coef_)
    predicted_values.append(y_pred)
    r2_scores.append(r2_score(y_test, y_pred))

    print(f"Fold R2 Score: {r2_scores[-1]}")
    print(f"Fold Beta Matrix: {beta_matrices[-1]}")
    print("-" * 20)

print("\nAverage R2 Score:", np.mean(r2_scores))

Fold R2 Score: 0.9179971706985147
Fold Beta Matrix: [230745.94073479 163243.27314515 120309.77397759   3011.45976111
 151552.63069359]
--------------------
Fold R2 Score: 0.9145677884802819
Fold Beta Matrix: [229081.97914235 165882.1605634  121536.57475055   2092.4478622
 150874.99274586]
--------------------
Fold R2 Score: 0.9116116385364478
Fold Beta Matrix: [230224.50511001 162766.17455493 121022.77324577   1247.16258975
 150234.77720419]
--------------------
Fold R2 Score: 0.9193091764960816
Fold Beta Matrix: [229500.10043209 165212.07110924 122839.9376815    3063.71699324
 150917.88484984]
--------------------
Fold R2 Score: 0.9243869413350316
Fold Beta Matrix: [230225.0513193  163956.83884606 121115.12045628    783.46716975
 150662.44678192]
--------------------

Average R2 Score: 0.9175745431092714


e) Use the best value of (𝛽) matrix (for which R2_score is maximum), to train the regressor for 70% of data and test the performance for remaining 30% data.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

# Find the index of the maximum R2 score
best_r2_index = np.argmax(r2_scores)
best_beta = beta_matrices[best_r2_index]

# Split the data into 70% training and 30% testing
x_train_final, x_test_final, y_train_final, y_test_final = train_test_split(x, y, test_size=0.3, random_state=42)

# Create a new LinearRegression model and train it on the 70% data
final_model = LinearRegression()
final_model.fit(x_train_final, y_train_final)

# Make predictions on the test set
y_pred_final = final_model.predict(x_test_final)

# Evaluate the model
final_r2_score = r2_score(y_test_final, y_pred_final)

print(f"R2 Score on 70/30 split using model trained on 70% data: {final_r2_score}")
print(f"Beta Matrix from model trained on 70% data: {final_model.coef_}")

R2 Score on 70/30 split using model trained on 70% data: 0.9146818498916267
Beta Matrix from model trained on 70% data: [230464.52520478 164159.19982569 120514.71328324   2913.62424674
 151019.35865135]


**Q2.** Concept of Validation set for Multiple Linear Regression (Gradient Descent  
Optimization).Consider the same dataset of Q1, rather than dividing the dataset into five folds, divide the
dataset into training set (56%), validation set (14%), and test set (30%). Consider four different values of learning rate i.e. {0.001,0.01,0.1,1}. Compute the values of
regression coefficients for each value of learning rate after 1000 iterations. For each set of regression coefficients, compute R2_score for validation and test set and find the best value of regession coefficient.

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

data = pd.read_csv("USA_Housing.csv")
X = data.drop(columns=['Price']).values
y = data['Price'].values.reshape(-1, 1)

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.20, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

def add_bias(X):
    return np.c_[np.ones((X.shape[0], 1)), X]

X_train_b = add_bias(X_train)
X_val_b = add_bias(X_val)
X_test_b = add_bias(X_test)

def gradient_descent(X, y, lr, iterations=1000):
    m, n = X.shape
    beta = np.zeros((n, 1))
    for _ in range(iterations):
        gradient = (1/m) * (X.T @ (X @ beta - y))
        beta -= lr * gradient
    return beta

learning_rates = [0.001, 0.01, 0.1, 1]
best_r2_val = -np.inf
best_beta = None
best_lr = None

for lr in learning_rates:
    beta = gradient_descent(X_train_b, y_train, lr, iterations=1000)
    y_val_pred = X_val_b @ beta
    y_test_pred = X_test_b @ beta
    r2_val = r2_score(y_val, y_val_pred)
    r2_test = r2_score(y_test, y_test_pred)
    print(f"Learning Rate = {lr}")
    print("R² on Validation Set:", r2_val)
    print("R² on Test Set:", r2_test)
    print("Beta Coefficients:\n", beta.flatten()[:5], "...")
    print("-"*40)
    if r2_val > best_r2_val:
        best_r2_val = r2_val
        best_beta = beta
        best_lr = lr

print("\nBest Learning Rate:", best_lr)
print("Best Validation R²:", best_r2_val)
print("Best Beta Coefficients:\n", best_beta.flatten())


Learning Rate = 0.001
R² on Validation Set: -0.799334149357984
R² on Test Set: -0.9872590714052005
Beta Coefficients:
 [779274.02090139 146658.52639308 103040.26919133  72303.5523883
  24381.66647824] ...
----------------------------------------
Learning Rate = 0.01
R² on Validation Set: 0.909818561206292
R² on Test Set: 0.9147438859798211
Beta Coefficients:
 [1232381.36998704  234542.29331285  162395.51342821  121485.70570465
    3089.76469737] ...
----------------------------------------
Learning Rate = 0.1
R² on Validation Set: 0.9097995626742029
R² on Test Set: 0.9147570103083724
Beta Coefficients:
 [1232434.57572502  234562.99454568  162415.95445045  121760.31153933
    2814.16200824] ...
----------------------------------------
Learning Rate = 1
R² on Validation Set: 0.9097995626742028
R² on Test Set: 0.9147570103083724
Beta Coefficients:
 [1232434.57572502  234562.99454568  162415.95445045  121760.31153933
    2814.16200824] ...
----------------------------------------

Best Lea

Pre-processing and Multiple Linear Regression  
Download the dataset regarding Car Price Prediction from the following link:  
https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data  
1. Load the dataset with following column names ["symboling", "normalized_losses",  
"make", "fuel_type", "aspiration","num_doors", "body_style", "drive_wheels",  
"engine_location", "wheel_base", "length", "width", "height", "curb_weight",  
"engine_type", "num_cylinders", "engine_size", "fuel_system", "bore", "stroke",  
"compression_ratio", "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"]  
and replace all ? values with NaN  
2. Replace all NaN values with central tendency imputation. Drop the rows with NaN  
values in price column  
3. There are 10 columns in the dataset with non-numeric values. Convert these values to  
numeric values using following scheme:  
(i) For “num_doors” and “num_cylinders”: convert words (number names) to figures  
for e.g., two to 2  
(ii) For "body_style", "drive_wheels": use dummy encoding scheme  
(iii) For “make”, “aspiration”, “engine_location”,fuel_type: use label encoding  
scheme  
(iv) For fuel_system: replace values containing string pfi to 1 else all values to 0.  
(v) For engine_type: replace values containing string ohc to 1 else all values to 0.  
4. Divide the dataset into input features (all columns except price) and output variable  
(price). Scale all input features.  
5. Train a linear regressor on 70% of data (using inbuilt linear regression function of  
Python) and test its performance on remaining 30% of data.  
6. Reduce the dimensionality of the feature set using inbuilt PCA decomposition and then  
again train a linear regressor on 70% of reduced data (using inbuilt linear regression function of python. Does it lead to any perfromance improvement on the test set?

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.decomposition import PCA

columns = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location", "wheel_base",
           "length", "width", "height", "curb_weight", "engine_type", "num_cylinders",
           "engine_size", "fuel_system", "bore", "stroke", "compression_ratio",
           "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"]

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"
data = pd.read_csv(url, names=columns)

data = data.replace("?", np.nan)

# Central tendency imputation
for col in data.columns:
    if data[col].dtype == "object":
        mode_value = data[col].mode()[0]
        data[col] = data[col].fillna(mode_value)
    else:
        data[col] = pd.to_numeric(data[col], errors="coerce")
        median_value = data[col].median()
        data[col] = data[col].fillna(median_value)

# Drop rows where price is NaN
data = data.dropna(subset=["price"])
data["price"] = data["price"].astype(float)

# Encoding
num_map = {"two": 2, "three": 3, "four": 4, "five": 5,
           "six": 6, "eight": 8, "twelve": 12}
data["num_doors"] = data["num_doors"].replace(num_map).astype(int)
data["num_cylinders"] = data["num_cylinders"].replace(num_map).astype(int)

data = pd.get_dummies(data, columns=["body_style", "drive_wheels"], drop_first=True)

for col in ["make", "aspiration", "engine_location", "fuel_type"]:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])

data["fuel_system"] = data["fuel_system"].apply(lambda x: 1 if "pfi" in x else 0)
data["engine_type"] = data["engine_type"].apply(lambda x: 1 if "ohc" in x else 0)

X = data.drop(columns=["price"]).values
y = data["price"].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

r2_original = r2_score(y_test, y_pred)
mse_original = mean_squared_error(y_test, y_pred)

print("Linear Regression without PCA")
print("R² Score:", r2_original)
print("MSE:", mse_original)

pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

Xp_train, Xp_test, yp_train, yp_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

lr_pca = LinearRegression()
lr_pca.fit(Xp_train, yp_train)
yp_pred = lr_pca.predict(Xp_test)

r2_pca = r2_score(yp_test, yp_pred)
mse_pca = mean_squared_error(yp_test, yp_pred)

print("\nLinear Regression with PCA")
print("R² Score:", r2_pca)
print("MSE:", mse_pca)

if r2_pca > r2_original:
    print("\n✅ PCA improved performance")
else:
    print("\n❌ PCA did not improve performance")


Linear Regression without PCA
R² Score: 0.7895045576733848
MSE: 14448999.011837844

Linear Regression with PCA
R² Score: 0.7478420860380317
MSE: 17308828.207359694

❌ PCA did not improve performance


/tmp/ipython-input-949230477.py:37: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["num_doors"] = data["num_doors"].replace(num_map).astype(int)
/tmp/ipython-input-949230477.py:38: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["num_cylinders"] = data["num_cylinders"].replace(num_map).astype(int)
